In [1]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import statsmodels.api as sm
from statsmodels.formula.api import ols
import seaborn as sns
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE
import warnings
warnings.filterwarnings("ignore")

covid_df = pd.read_csv('multivariate_eng_df.csv')
covid_df

,Unnamed: 0,Date,STATE,PROVINCE_STATE_NAME,Completeness_pct,Administered_Dose1_Recip,Administered_Dose1_Recip_5Plus,Administered_Dose1_Recip_12Plus,Administered_Dose1_Recip_18Plus,Administered_Dose1_Recip_65Plus,...,vaccination_rate_complete,booster_rate,prop_age04,prop_age59,prop_age6064,prop_age8084,prop_age85plus,AGE04_29_TOT,AGE30_59_TOT,AGE60PLUS_TOT
0,0,2020-01-21,1,Alabama,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,5.810037,5.990525,6.557529,2.031935,1.845802,1622745,1901707,1205505
1,1,2020-01-22,1,Alabama,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,5.810037,5.990525,6.557529,2.031935,1.845802,1622745,1901707,1205505
2,2,2020-01-23,1,Alabama,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,5.810037,5.990525,6.557529,2.031935,1.845802,1622745,1901707,1205505
3,3,2020-01-24,1,Alabama,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,5.810037,5.990525,6.557529,2.031935,1.845802,1622745,1901707,1205505
4,4,2020-01-25,1,Alabama,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,5.810037,5.990525,6.557529,2.031935,1.845802,1622745,1901707,1205505
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41495,41495,2022-04-25,56,Wyoming,2233.3,329321.0,329281.0,320164.0,301436.0,89872.0,...,0.497625,0.213973,5.390613,6.173920,6.798296,1.940552,1.789704,183567,214337,147583
41496,41496,2022-04-26,56,Wyoming,2233.3,329436.0,329396.0,320270.0,301530.0,89903.0,...,0.497813,0.214267,5.390613,6.173920,6.798296,1.940552,1.789704,183567,214337,147583
41497,41497,2022-04-27,56,Wyoming,2233.3,329503.0,329463.0,320324.0,301580.0,89919.0,...,0.497939,0.214424,5.390613,6.173920,6.798296,1.940552,1.789704,183567,214337,147583
41498,41498,2022-04-28,56,Wyoming,2233.3,329578.0,329538.0,320393.0,301639.0,89944.0,...,0.498093,0.214666,5.390613,6.173920,6.798296,1.940552,1.789704,183567,214337,147583


In [4]:
threshold_date = '2020-03-01'

# Convert the 'Date' column to datetime if it's not already
covid_df['Date'] = pd.to_datetime(covid_df['Date'])

# Filter the DataFrame to remove observations prior to the threshold date
covid_df = covid_df[covid_df['Date'] >= threshold_date]

columns_to_drop_1 = ['Unnamed: 0', 'STATE']
covid_df = covid_df.drop(columns=columns_to_drop_1)

In [6]:
covid_df_top_10 = covid_df.loc[covid_df['PROVINCE_STATE_NAME'].isin(['California', 'Texas', 'Florida', 'New York', 'Pennsylvania', 'Illinois', 'Ohio', 'Georgia', 'North Carolina', 'Michigan'])]
covid_df_top_10

,Date,PROVINCE_STATE_NAME,Completeness_pct,Administered_Dose1_Recip,Administered_Dose1_Recip_5Plus,Administered_Dose1_Recip_12Plus,Administered_Dose1_Recip_18Plus,Administered_Dose1_Recip_65Plus,Series_Complete_Yes,Series_Complete_5Plus,...,vaccination_rate_complete,booster_rate,prop_age04,prop_age59,prop_age6064,prop_age8084,prop_age85plus,AGE04_29_TOT,AGE30_59_TOT,AGE60PLUS_TOT
3360,2020-03-01,California,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,5.744418,6.165309,5.818187,1.716470,1.860398,13077768,15818572,8169914
3361,2020-03-02,California,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,5.744418,6.165309,5.818187,1.716470,1.860398,13077768,15818572,8169914
3362,2020-03-03,California,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,5.744418,6.165309,5.818187,1.716470,1.860398,13077768,15818572,8169914
3363,2020-03-04,California,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,5.744418,6.165309,5.818187,1.716470,1.860398,13077768,15818572,8169914
3364,2020-03-05,California,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,5.744418,6.165309,5.818187,1.716470,1.860398,13077768,15818572,8169914
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35685,2022-04-25,Texas,25120.6,20890120.0,20884105.0,19864532.0,18040473.0,3628247.0,17587233.0,17585807.0,...,0.585664,0.219948,6.335885,6.946010,5.387383,1.469608,1.337448,10530755,11768099,5644861
35686,2022-04-26,Texas,25120.6,20897363.0,20891339.0,19869792.0,18044864.0,3628950.0,17591452.0,17590025.0,...,0.585804,0.221604,6.335885,6.946010,5.387383,1.469608,1.337448,10530755,11768099,5644861
35687,2022-04-27,Texas,25120.6,20907738.0,20901703.0,19877395.0,18051163.0,3630034.0,17598026.0,17596598.0,...,0.586023,0.221844,6.335885,6.946010,5.387383,1.469608,1.337448,10530755,11768099,5644861
35688,2022-04-28,Texas,25120.6,20914938.0,20908897.0,19883027.0,18056005.0,3631069.0,17603464.0,17602036.0,...,0.586204,0.222099,6.335885,6.946010,5.387383,1.469608,1.337448,10530755,11768099,5644861


In [8]:
univariate_df = covid_df_top_10.loc[:, ['Date', 'PROVINCE_STATE_NAME', 'PEOPLE_POSITIVE_NEW_CASES_COUNT']]
univariate_df

,Date,PROVINCE_STATE_NAME,PEOPLE_POSITIVE_NEW_CASES_COUNT
3360,2020-03-01,California,5
3361,2020-03-02,California,5
3362,2020-03-03,California,7
3363,2020-03-04,California,10
3364,2020-03-05,California,12
...,...,...,...
35685,2022-04-25,Texas,1243
35686,2022-04-26,Texas,2556
35687,2022-04-27,Texas,2513
35688,2022-04-28,Texas,2715


In [9]:
cali_uni_df = univariate_df.loc[covid_df['PROVINCE_STATE_NAME'].isin(['California'])]
texas_uni_df = univariate_df.loc[covid_df['PROVINCE_STATE_NAME'].isin(['Texas'])]
florida_uni_df = univariate_df.loc[covid_df['PROVINCE_STATE_NAME'].isin(['Florida'])]
ny_uni_df = univariate_df.loc[covid_df['PROVINCE_STATE_NAME'].isin(['New York'])]
penn_uni_df = univariate_df.loc[covid_df['PROVINCE_STATE_NAME'].isin(['Pennsylvania'])]
illinois_uni_df = univariate_df.loc[covid_df['PROVINCE_STATE_NAME'].isin(['Illinois'])]
ohio_uni_df = univariate_df.loc[covid_df['PROVINCE_STATE_NAME'].isin(['Ohio'])]
georgia_uni_df = univariate_df.loc[covid_df['PROVINCE_STATE_NAME'].isin(['Georgia'])]
nc_uni_df = univariate_df.loc[covid_df['PROVINCE_STATE_NAME'].isin(['North Carolina'])]
mich_uni_df = univariate_df.loc[covid_df['PROVINCE_STATE_NAME'].isin(['Michigan'])]

In [11]:
cali_uni_df.to_csv('california_uni.csv', index=False)
texas_uni_df.to_csv('texas_uni.csv', index=False)
florida_uni_df.to_csv('florida_uni.csv', index=False)
ny_uni_df.to_csv('newyork_uni.csv', index=False)
penn_uni_df.to_csv('penn_uni.csv', index=False)
illinois_uni_df.to_csv('illinois_uni.csv', index=False)
ohio_uni_df.to_csv('ohio_uni.csv', index=False)
georgia_uni_df.to_csv('georgia_uni.csv', index=False)
nc_uni_df.to_csv('northcarolina_uni.csv', index=False)
mich_uni_df.to_csv('michigan_uni.csv', index=False)